## 数据增广
- 很有必要，比如色温，亮度等的变化是很容易产生的，但是模型本身不应该对这些因素非常敏感
- 增加一个已有数据集，使得具有更多的多样性
- 数据增广一般是`在线生成`， 就是数据集读进去之后再数据增广
- 增广在有些情况下是为了去逼近测试集

- 高斯滤波分为高通滤波和低通滤波，高通滤波主要作用是锐化， 低通滤波主要作用是平滑

### 方法
1. 反转
   1. 上下翻转
   2. 左右翻转
2. 切割
   1. 切割出一块然后再变成固定形状
   2. 随机高宽比(e.g.[3/4, 4/3])
   3. 随机大小(e.g.[8%, 100%])
   4. 随机位置
3. 颜色
   1. 色调
   2. 饱和度
   3. 明亮度(e.g.[0.5, 1.5])